In [ ]:
# %pip install PySuperTuxKart==1.1.2
# # !pip install imageio==2.4.1
# !pip install imageio

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


# Setup

In [ ]:
import os
os._exit(00)

In [ ]:
# Environmental flag to visualize in colab.


%env ON_COLAB=1
from importlib import reload
import numpy as np
import torch
from torch.nn import *
import torch.nn.functional as F

from modules_official import dense_transforms;reload(dense_transforms)
from modules_official import controller;reload(controller)
from modules_official import planner;reload(planner)
from modules_official import train;reload(train)
from modules_official import util;reload(util)


from modules_jt import control_func_module_jt;reload(control_func_module_jt)
from modules_jt import generate_dataset_module_jt;reload(generate_dataset_module_jt)
from modules_jt import train_module_jt;reload(train_module_jt)
from modules_jt import train_module_jt2;reload(train_module_jt2)
from modules_jt import global_variables;reload(global_variables)

from adaption_pkg import train_adaption_mod;reload(train_adaption_mod)

from modules_team import control_func_module_team



if 'pytux_jt' not in locals():
    pytux_jt = util.PyTuxJT()



env: ON_COLAB=1


In [ ]:
class ClassifierJT(torch.nn.Module):
  def __init__(self):

    super().__init__()
    self.batch_norm=BatchNorm2d(num_features=3)
    self.resnet=torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)

    post_resnet=[]
    post_resnet.append(ReLU(inplace=True))
    post_resnet.append(Linear(in_features=1000,out_features=global_variables.NUM_TRACKS))
    self.post_resnet=torch.nn.Sequential(*post_resnet)
    
  def forward(self, image_batch):# [batch_size,channel_size=3,height=96,width=128]
    """
    Your code here
    Predict the aim point in image coordinate, given the supertuxkart image
    @img: (B,3,96,128)
    return (B,2) 
    """
    x = self.batch_norm(image_batch)
    x = self.resnet(x)#[batch_size, 1, 6, 8]
    x = self.post_resnet(x)
    
    return x



def spatial_argmax_jt(logit):
    """
    Compute the soft-argmax of a heatmap
    :param logit: A tensor of size BS x H x W
    :return: A tensor of size BS x 2 the soft-argmax in normalized coordinates (-1 .. 1)
    """
   
   
    weights = F.softmax(logit.view(logit.size(0), -1), dim=-1).view_as(logit)
    #ok, let's discuss what this just did
    # logit.view(logit.size(0), -1) reshapes the input, which is a 1xaxb tensor as 
    # 1 x ab
    #softmax then takes the softmax
    #finally, this is converted back to a 1xaxb tensor

    #the following code computes the center of mass
    #formula in more convenient form is given in slides
    # print(weights[0])
    # linspace(-1,1)是一個數列，第一個點為-1，最後一個點為 1，一共有 logit.shape[2]個數。類似於找重心。
    firstcoord = (weights.sum(1) * torch.linspace(-1, 1, logit.size(2)).to(logit.device)[None]).sum(1)
    secondcoord = (weights.sum(2) * torch.linspace(-1, 1, logit.size(1)).to(logit.device)[None]).sum(1)
    a = torch.stack((firstcoord, secondcoord), 1)
    

    return a


class PlannerResJT(torch.nn.Module):
  def __init__(self):

    super().__init__()
    self.batch_norm=BatchNorm2d(num_features=3)
    self.resnet=torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
    

  def forward(self, image_batch):# [batch_size,channel_size=3,height=96,width=128]
    """
    Your code here
    Predict the aim point in image coordinate, given the supertuxkart image
    @img: (B,3,96,128)
    return (B,2) 
    """
    
    x = self.batch_norm(image_batch)
    x = self.resnet(x)#[batch_size, 1, 6, 8]
    x=x.reshape(-1,25,40)
    # print(x.shape)

    return spatial_argmax_jt(x) #輸入是[batch_size,height=6,width=8],輸出是[batch_size,2]

#### M3

In [ ]:
classifier=train_module_jt.load_planner_jt("classifier_res_jt1_v3.th", ClassifierJT)
classifier.eval()

class AdaptedPlannerM3JT(torch.nn.Module):
  def __init__(self,device="cuda:0"):
    super().__init__()

    self.device=device
    self.batch_norm=BatchNorm2d(num_features=3)
    self.resnet=torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
    

    self.adapters=ModuleList()
    for track_code in range(global_variables.NUM_TRACKS):
      
      ll6=[]
      ll=ll6
      
      ll.append(Linear(in_features=1000,out_features=1024))
      ll.append(ReLU(inplace=True))
      ll.append(Linear(1024,2048))
      ll.append(ReLU(inplace=True))
      ll.append(Linear(2048,1024))
      ll.append(ReLU(inplace=True))
      ll.append(Linear(1024,512))
      ll.append(ReLU(inplace=True))
      ll.append(Linear(512,256))
      ll.append(ReLU(inplace=True))
      ll.append(Linear(256,128))
      ll.append(ReLU(inplace=True))
      ll.append(Linear(128,40))
      adapter=torch.nn.Sequential(*ll6)
      self.adapters.append(adapter)

  def forward(self, image_batch, code_batch=None,device=None):# [batch_size,channel_size=3,height=96,width=128]
    
    if device is None:
      device=self.device

    if code_batch is None:
      classifier.to(device)
      classifier.eval()
      code_batch=classifier(image_batch).argmax(1)

    
    
    
    x = self.batch_norm(image_batch)
    x = self.resnet(x)#[batch_size, 1000]


    batch_size=len(image_batch)
    num_tracks=global_variables.NUM_TRACKS
    width=40
    height=25
    


    indices_list=[[] for i in range(num_tracks)]
    for index in range(batch_size):
      indices_list[code_batch[index].item()].append(index) 




    mini_map_batches=[]
    for i in range(num_tracks):
      mini_map_batches.append(x.index_select(0,torch.tensor(indices_list[i], dtype=torch.int32).to(device)))
    

    weights_batch=[1]*batch_size
    for i in range(num_tracks):

      mini_map_batch=mini_map_batches[i]
      mini_map_batch.to(device)

      if len(mini_map_batch)==0:
        continue

      z=self.adapters[i](mini_map_batch) # batch_size, 40
      z=F.softmax(z, dim=1)


      for j in range(len(mini_map_batch)):
        weights_batch[indices_list[i][j]]=z[j]
    
    weights_batch=torch.stack(weights_batch).to(device) # batchsize, 1, 40
    x=x.reshape(-1,25,40)

    x=x*weights_batch.unsqueeze(1)


  
    return spatial_argmax_jt(x)


# adapted_planner_m2=AdaptedPlannerM2JT(device="cpu")
# print(adapted_planner_m2)
# output=adapted_planner_m2(image_batch.to("cpu"),code_batch=code_batch.to("cpu"),device="cpu")
# print(output)


Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# New Section

#### train model

In [ ]:
args_jt=train_module_jt2.TrainArgsJT2()
args_jt.num_workers=2
args_jt.random_seed=99998
tracks = [
    'lighthouse', 
    'zengarden', 
    'hacienda', 
    'snowtuxpeak',
    'cornfield_crossing',
    'scotland',
    'cocoa_temple'
          ]
args_jt.tracks=tracks

args_jt.trainset_dirs=[
    'trainset_5000_samjt',
    'trainset_5000_charlie',
    # 'trainset_5000_jt'
          ]
args_jt.valset_dirs=[
    'valset_5000_samjt',
    'valset_5000_charlie',
    # 'valset_5000_jt'
          ]
args_jt.device="cuda:3"
args_jt.learning_rate=8e-3
args_jt.val_interval=1
args_jt.num_epochs=5000
args_jt.proportion=1

args_jt.planner_name="adapted_planner_m3_jt3_v1.th";
args_jt.planner_type=AdaptedPlannerM3JT

train_adaption_mod.train_adaption_jt(args_jt)

Initializing Model =================================


Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0


AdaptedPlannerM3JT(
  (batch_norm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

#### AdaptedPlannerM2JT

In [ ]:


class Recognizer(PlannerResJT):
  def forward(self, image_batch):# [batch_size,channel_size=3,height=96,width=128]
    """
    Your code here
    Predict the aim point in image coordinate, given the supertuxkart image
    @img: (B,3,96,128)
    return (B,2) 
    """
    
    x = self.batch_norm(image_batch)
    x = self.resnet(x)#[batch_size, 1, 6, 8]
    return x 



recognizer=train_module_jt.load_planner_jt("planner_res_jt_v2.th", Recognizer)
recognizer.eval()

classifier=train_module_jt.load_planner_jt("classifier_res_jt1_v3.th", ClassifierJT)
classifier.eval()




class AdaptedPlannerM2JT(torch.nn.Module):
  def __init__(self,device="cuda:0"):

    super().__init__()
    

    self.adapters=ModuleList()
    for track_code in range(global_variables.NUM_TRACKS):
      
      ll5=[]
      ll=ll5
      ll.append(BatchNorm2d(num_features=1))
      ll.append(Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1))
      ll.append(ReLU(inplace=True))
      ll.append(Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1))
      ll.append(ReLU(inplace=True))

      # ll.append(MaxPool2d(kernel_size=2,stride=2))

      ll.append(BatchNorm2d(num_features=32))
      ll.append(Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=2))
      ll.append(ReLU(inplace=True))
      ll.append(Conv2d(in_channels=64,out_channels=64,kernel_size=5,stride=1,padding=2))
      ll.append(ReLU(inplace=True))
      ll.append(Conv2d(in_channels=64,out_channels=64,kernel_size=5,stride=1,padding=2))
      ll.append(ReLU(inplace=True))

      # ll.append(MaxPool2d(kernel_size=2,stride=2))

      ll.append(BatchNorm2d(num_features=64))
      ll.append(Conv2d(in_channels=64,out_channels=64,kernel_size=5,stride=1,padding=2))
      ll.append(ReLU(inplace=True))

      ll.append(Conv2d(in_channels=64,out_channels=1,kernel_size=1))


      adapter=torch.nn.Sequential(*ll5)
      self.adapters.append(adapter)
    
    
  def forward(self, image_batch, code_batch=None,device="cuda:0"):# [batch_size,channel_size=3,height=96,width=128]
    """
    Your code here
    Predict the aim point in image coordinate, given the supertuxkart image
    @img: (B,3,96,128)
    return (B,2) 
    """

    if code_batch is None:
      classifier.to(device)
      classifier.eval()
      code_batch=classifier(image_batch).argmax(1)


    batch_size=len(image_batch)
    num_tracks=global_variables.NUM_TRACKS
    width=40
    height=25
    


    indices_list=[[] for i in range(num_tracks)]
    for index in range(batch_size):
      indices_list[code_batch[index].item()].append(index) 


    recognizer.to(device)
    recognizer.eval()
    map_batch=recognizer(image_batch).reshape(-1,1,height,width)



    mini_map_batches=[]
    for i in range(num_tracks):
      mini_map_batches.append(map_batch.index_select(0,torch.tensor(indices_list[i], dtype=torch.int32).to(device)))
    

    weights_batch=[1]*batch_size
    for i in range(num_tracks):

      mini_map_batch=mini_map_batches[i]
      mini_map_batch.to(device)

      if len(mini_map_batch)==0:
        continue

      z=self.adapters[i](mini_map_batch)

      for j in range(len(mini_map_batch)):
        weights_batch[indices_list[i][j]]=z[j]
    


    # weights_batch=F.softmax(weights_batch, dim=1)
    weights_batch=torch.stack(weights_batch).to(device) # batchsize, 1, 40


    return spatial_argmax_jt(weights_batch[:,0])


# adapted_planner_m2=AdaptedPlannerM2JT(device="cpu")
# print(adapted_planner_m2)
# output=adapted_planner_m2(image_batch.to("cpu"),code_batch=code_batch.to("cpu"),device="cpu")
# print(output)



Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0


#### AdaptedPlannerJT

In [ ]:



classifier=train_module_jt.load_planner_jt("classifier_res_jt1_v3.th", ClassifierJT)
classifier.eval()





class AdaptedPlannerJT(torch.nn.Module):
  def __init__(self,device="cuda:0"):

    super().__init__()
    
    if True:
      self.pool=torch.nn.AvgPool2d(kernel_size=3, stride=3,padding=1)
      batch_norms=[]
      adapters=[]
      for track_code in range(global_variables.NUM_TRACKS):
        batch_norms.append(BatchNorm2d(num_features=3).to(device))
        adapters.append(torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False).to(device))
      self.batch_norms=ModuleList(batch_norms)
      self.adapters=ModuleList(adapters)


    self.main_batch_norm=BatchNorm2d(num_features=3).to(device)
    self.resnet=torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False).to(device)
    

  def forward(self, image_batch, code_batch=None,device="cuda:0"):# [batch_size,channel_size=3,height=96,width=128]
    """
    Your code here
    Predict the aim point in image coordinate, given the supertuxkart image
    @img: (B,3,96,128)
    return (B,2) 
    """

    if True:

      if code_batch is None:
        classifier.to(device)
        classifier.eval()
        code_batch=classifier(image_batch).argmax(1)

      batch_size=len(image_batch)
      num_tracks=global_variables.NUM_TRACKS

      indices_list=[[] for i in range(num_tracks)]
      for index in range(batch_size):
        indices_list[code_batch[index].item()].append(index) 
  

      mini_batches=[]
      for i in range(num_tracks):

        mini_batches.append(image_batch.index_select(0,torch.tensor(indices_list[i], dtype=torch.int32).to(device)))
      
      adapted_batch=torch.zeros(batch_size,1000).to(device)

      for i in range(num_tracks):

        mini_batch=mini_batches[i]
        mini_batch.to(device)

        if len(mini_batch)==0:
          continue
        
        z=self.pool(mini_batch)
        z=self.batch_norms[i](z)
        z=self.adapters[i](z)
        for j in range(len(mini_batch)):
          adapted_batch[indices_list[i][j]]=z[j]

      # adapted_batch=F.softmax(F.softmax(adapted_batch, dim=1)*1000,dim=1)*1000
      adapted_batch=F.softmax(F.softmax(adapted_batch, dim=1)*1000,dim=1)*1000

      adapted_batch.to(device)
      # print("adapted_batch",adapted_batch)
      # print(adapted_batch.sum(1))

    x = self.main_batch_norm(image_batch)
    x = self.resnet(x)
    x.to(device)


    x=x.mul(adapted_batch) #jt5 adapted

    x=x.reshape(-1,25,40)


    return spatial_argmax_jt(x) #輸入是[batch_size,height=6,width=8],輸出是[batch_size,2]



adapted_planner=AdaptedPlannerJT(device="cpu")
output=adapted_planner(image_batch.to("cpu"),code_batch=code_batch.to("cpu"),device="cpu")
print(output)
# print(code_batch.index_select(0,torch.tensor([0,1,2])))

Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/phuang/.cache/torch/hub/pytorch_vision_v0.10.0


tensor([[-0.2308, -0.2500],
        [ 0.2248,  0.0163]], grad_fn=<StackBackward0>)


#### test planner

In [ ]:
def test_planner_jt(pytux,tracks, planner_name,planner_type, control_func,verbose):
    # Load model
    planner = train_module_jt.load_planner_jt(planner_name,planner_type).eval()
    print(planner)
    planner.to("cuda")

    sum=0
    for t in tracks:
        steps, how_far = pytux.rollout(t, control_func, planner=planner, max_frames=1000, verbose=verbose)
        sum=sum+steps
        print(t,steps, how_far)
    print("total",sum)


tracks = [
    'lighthouse', 
    'zengarden', 
    'hacienda', 
    'snowtuxpeak',
    'cornfield_crossing',
    'scotland',
    'cocoa_temple'
          ]   

test_planner_jt(pytux_jt,
                tracks,
                planner_name="adapted_planner_jt1_v1.th",
                planner_type=AdaptedPlannerJT,
                control_func=control_func_module_team.control_func_samjt,
                verbose=True)


In [ ]:
def get_demo_images_codes_batch():

    dataset_path="dataset_base/valset_5000_samjt/zengarden"
    transform=dense_transforms.ToTensor()

    data = []
    NUM=2
    counter=NUM
    from PIL import Image
    from glob import glob
    from os import path

    for f in glob(path.join(dataset_path, '*.csv')):

      i = Image.open(f.replace('.csv', '.png'))
      i.load()
      data.append((i, np.loadtxt(f, dtype=np.float32, delimiter=',')))

      counter-=1
      if counter==0:
        break

    image_list=[]
    code_list=[0]*NUM
    for idx in range(NUM):
      sample = data[idx]
      sample = transform(*sample)
      image, label= sample
      image_list.append(image)

    image_batch=torch.stack(image_list)
    code_batch=torch.tensor(code_list,dtype=torch.int)
    return image_batch, code_batch



In [ ]:
image_batch, code_batch=get_demo_images_codes_batch()
print(image_batch.shape)
print(code_batch.shape)


torch.Size([2, 3, 96, 128])
torch.Size([2])


In [ ]:
image_batch_c=image_batch.clone()

classifier_jt=train_module_jt.load_planner_jt("classifier_res_jt1_v3.th", ClassifierJT)
classifier_jt.eval()
print(classifier_jt(image_batch).argmax(1))

print(torch.all(torch.eq(image_batch, image_batch_c)))



NameError: ignored